---
对于线性模型
$$y=\beta_0+\beta_1x_1+\cdots+\beta_mx_m+\varepsilon$$
令 $\mu=E(y)$，则
$$\mu=\beta_0+\beta_1x_1+\cdots+\beta_mx_m$$
广义线性模型 (GLM) 将上式推广为
$$\eta=g(\mu)=\beta_0+\beta_1x_1+\cdots+\beta_mx_m$$
故有如下三个概念：
1. 线性自变量：$\beta_0+\beta_1x_1+\cdots+\beta_mx_m$
2. 连接函数：$\eta=g(\mu)$
3. 误差函数，即模型的随机部分，可从指数型分布族中选择误差函数

常见的连接函数和误差函数
|           |       连接函数        |         逆连接函数（回归模型）         | 典型误差函数  |
| :-------: | :-------------------: | :------------------------------------: | :-----------: |
| **恒等**  |      $\eta=\mu$       |               $\mu=\eta$               |   正态分布    |
| **对数**  |     $\eta=\ln\mu$     |            $\mu=\exp(\eta)$            |   泊松分布    |
| **Logit** | $\eta=\rm logit \mu$  | $\mu=\dfrac{\exp(\eta)}{1+\exp(\eta)}$ |   二项分布    |
|  **逆**   | $\eta=\dfrac{1}{\mu}$ |         $\mu=\dfrac{1}{\eta}$          | $\Gamma$ 分布 |

### 10.5.1 分组数据的 Logistic 回归模型
Logistic 回归方程为
$$p_i=\dfrac{{\rm e}^{\beta_0+\beta_1x_i}}{1+{\rm e}^{\beta_0+\beta_1x_i}}=\dfrac{1}{1+{\rm e}^{-\beta_0-\beta_1x_i}}$$
对以上回归方程进行线性化变换（这里为逻辑变换），令
$$p_i^*=\ln\left(\dfrac{p_i}{1-p_i}\right)$$
变换后的线性回归模型为
$$p^*=\beta_0+\beta_1x$$

In [1]:
import numpy as np
import statsmodels.api as sm

data = np.loadtxt("../../10第10章  回归分析/data10_7_1.txt")
x, n, m = data.T
p = m/n
y = np.log(p/(1-p))     # 进行逻辑变换
X = sm.add_constant(x)
mod = sm.OLS(y, X).fit()
print(mod.summary())

yhat = mod.predict([1, 9])
phat = np.exp(yhat)/(1+np.exp(yhat))    # 化回原方程
print("\n年收入为9万元的客户买房的比例为：", phat[0])

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.924
Model:                            OLS   Adj. R-squared:                  0.913
Method:                 Least Squares   F-statistic:                     85.42
Date:                Sun, 21 Aug 2022   Prob (F-statistic):           3.59e-05
Time:                        20:06:16   Log-Likelihood:                 6.6829
No. Observations:                   9   AIC:                            -9.366
Df Residuals:                       7   BIC:                            -8.971
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.8863      0.102     -8.653      0.0

/home/zl/anaconda3/envs/mamo/lib/python3.9/site-packages/scipy/stats/_stats_py.py:1769: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


#### glm 实现
```python
glm(formula, data, family=None)
```
`family` 的取值是分布类型，其中的分布可以为 `Binomial`、`Gamma`、`Gaussian`、`InverseGaussian`、`NegativeBinomial` 等

In [2]:
x = data[:, 0]
pos = data[:, 2]
neg = data[:, 1] - data[:, 2]
y = np.vstack([pos, neg]).T     # 第一列为成功的次数，第二列为失败的次数
mod_dic = {'x': x, 'y': y}
mod = sm.formula.glm('y~x', mod_dic, family=sm.families.Binomial()).fit()
print(mod.summary())
phat = mod.predict({'x': 9})[0]
print("\n年收入为9万元的客户买房的比例为：", phat)

                 Generalized Linear Model Regression Results                  
Dep. Variable:       ['y[0]', 'y[1]']   No. Observations:                    9
Model:                            GLM   Df Residuals:                        7
Model Family:                Binomial   Df Model:                            1
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -18.046
Date:                Sun, 21 Aug 2022   Deviance:                       1.0467
Time:                        20:06:16   Pearson chi2:                     1.05
No. Iterations:                     3   Pseudo R-squ. (CS):             0.5931
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.8518      0.293     -2.906      0.0

利用 `glm()` 求得的 Logistic 回归模型为
$$\hat p=\dfrac{1}{1+{\rm e}^{0.8518-0.1498x}}$$

### 10.5.2 未分组数据的 Logistic 回归模型

In [3]:
data = np.loadtxt('../../10第10章  回归分析/data10_8.txt')
data = np.vstack([data[:, :5], data[:, 5:]])
ind, x1, x2, x3, y = data.T
mod_dic = {'x1': x1, 'x2': x2, 'x3': x3, 'y':y}
mod = sm.formula.logit('y~x1+x2+x3', mod_dic).fit()
print(mod.summary())        # x2 的 Z 统计量小，不显著
print('\n\n\n')
mod1 = sm.formula.logit('y~x1+x3', mod_dic).fit()
print(mod1.summary())

Optimization terminated successfully.
         Current function value: 0.463762
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                   28
Model:                          Logit   Df Residuals:                       24
Method:                           MLE   Df Model:                            3
Date:                Sun, 21 Aug 2022   Pseudo R-squ.:                  0.3285
Time:                        20:06:16   Log-Likelihood:                -12.985
converged:                       True   LL-Null:                       -19.337
Covariance Type:            nonrobust   LLR p-value:                  0.005326
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -3.6550      2.091     -1.748      0.081      -7.754       0.444
x1             0.0822      0.

#### glm 实现

In [4]:
mod = sm.formula.glm('y~x1+x2+x3', mod_dic, family=sm.families.Binomial()).fit()
print(mod.summary())
print('\n\n\n')
mod1 = sm.formula.glm('y~x1+x3', mod_dic, family=sm.families.Binomial()).fit()
print(mod1.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                   28
Model:                            GLM   Df Residuals:                       24
Model Family:                Binomial   Df Model:                            3
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -12.985
Date:                Sun, 21 Aug 2022   Deviance:                       25.971
Time:                        20:06:16   Pearson chi2:                     29.1
No. Iterations:                     5   Pseudo R-squ. (CS):             0.3647
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -3.6550      2.091     -1.748      0.0

### 10.5.3 Probit 回归模型
假设随机扰动服从标准正态分布，回归函数为 
$$\Phi^{-1}(\pi_i)=\beta_0+\beta_1x_{i1}+\cdots+\beta_mx_{im}$$
其中 $\Phi(x)$ 为标准正态分布的分布函数。用样本比例 $p_i$ 替代概率 $\pi_i$，表示为样本回归模型
$$\Phi^{-1}(p_i)=\beta_0+\beta_1x_{i1}+\cdots+\beta_mx_{im}$$

In [5]:
from scipy.stats import norm

data = np.loadtxt('../../10第10章  回归分析/data10_7_1.txt')
x, n, m = data.T
p = m/n
y = norm.ppf(p)     # Probit 变换
X = sm.add_constant(x)
mod = sm.OLS(y, X).fit()
print(mod.summary())

yhat = mod.predict([1, 9])
phat = norm.cdf(yhat)
print("\n年收入为9万元的客户买房的比例为：", phat[0])

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.925
Model:                            OLS   Adj. R-squared:                  0.914
Method:                 Least Squares   F-statistic:                     86.12
Date:                Sun, 21 Aug 2022   Prob (F-statistic):           3.49e-05
Time:                        20:06:16   Log-Likelihood:                 10.983
No. Observations:                   9   AIC:                            -17.97
Df Residuals:                       7   BIC:                            -17.57
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.5518      0.064     -8.688      0.0

/home/zl/anaconda3/envs/mamo/lib/python3.9/site-packages/scipy/stats/_stats_py.py:1769: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


### 10.5.4 Logistic 回归模型的应用
#### 1. Logistic 模型的参数解释
一般地，如果 Logistic 模型
$$\ln{\dfrac{p}{1-p}}=\beta_0+\sum\limits_{j=1}^m\beta_jx_j$$
的参数估计为 $\hat\beta_0,\hat\beta_1,\cdots,\hat\beta_m$，则在 $x_1=x_{01},x_2=x_{02},\cdots,x_m=x_{0m}$ 条件下事件赔率 (odds) 的估计值为
$$\dfrac{\hat p_0}{1-\hat p_0}={\rm e}^{\hat\beta_0+\sum\limits_{j=1}^m\hat\beta_jx_{0j}}$$
如果记 $\boldsymbol X_A=\left[1,x_{A1},x_{A2},\cdots,x_{Am}\right]^{\rm T}$，$\boldsymbol X_B=\left[1,x_{B1},x_{B2},\cdots,x_{Bm}\right]^{\rm T}$，并将相应条件下的事件仍记为 $\boldsymbol X_A, \boldsymbol X_B$，则事件 $\boldsymbol X_A$ 对 $\boldsymbol X_B$ 赔率比的估计可由下式获得
$${\rm OR}\left(\boldsymbol X_A, \boldsymbol X_B\right)={\rm e}^{\sum\limits_{j=1}^m\hat\beta_j\left(x_{Aj}x_{Bj}\right)}$$

#### 2. Logistic 回归模型实例

In [6]:
data = np.loadtxt('../../10第10章  回归分析/data10_11.txt')
x1, x2, x3, y = data.T
mod_dic = {'x1': x1, 'x2': x2, 'x3': x3, 'y': y}
mod = sm.formula.logit('y~x1+x2+x3', mod_dic).fit(method='bfgs')    # 使用默认牛顿方法出错
print(mod.summary())

Optimization terminated successfully.
         Current function value: 0.000002
         Iterations: 18
         Function evaluations: 21
         Gradient evaluations: 21
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                   20
Model:                          Logit   Df Residuals:                       16
Method:                           MLE   Df Model:                            3
Date:                Sun, 21 Aug 2022   Pseudo R-squ.:                   1.000
Time:                        20:06:16   Log-Likelihood:            -4.6326e-05
converged:                       True   LL-Null:                       -13.863
Covariance Type:            nonrobust   LLR p-value:                 4.147e-06
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.3319   3.05e+04  -4.3

In [7]:
phat = mod.predict({'x1': [-49.2, 40.6], 'x2': [-17.2, 26.4], 'x3': [0.3, 1.8]})
phat

0    2.655616e-16
1    1.000000e+00
dtype: float64